In [ ]:
#optimiye tree topologz with alphago tzpe agent strategz


import gym
from torch_geometric.data import Data

# Define the Environment
class GraphEnvironment:
    def __init__(self):
        self.state = self.reset()
        self.action_dim = 4  # Number of nodes + 1 for the "end" action

    def reset(self):
        # Define the initial graph state
        x = torch.tensor([[1], [2], [3]], dtype=torch.float)  # Example node features
        edge_index = torch.tensor([[0, 1], [1, 2], [2, 0]], dtype=torch.long).t().contiguous()
        self.state = Data(x=x, edge_index=edge_index)
        return self.state

    def step(self, action):
        done = False
        if action == self.action_dim - 1:  # "end" action
            done = True
            reward = -torch.sum(self.state.x).item()  # Example reward function
        else:
            # Modify the graph based on the action
            self.state.x[action] += 1
            reward = -torch.sum(self.state.x).item()  # Example reward function
            done = torch.sum(self.state.x) > 10  # Example termination condition
        return self.state, reward, done


env = GraphEnvironment()
input_dim = env.reset().num_node_features
hidden_dim = 64
output_dim = hidden_dim
action_dim = env.action_dim  # Number of nodes + 1 for the "end" action



In [ ]:
#create a graph environment class for phylogenetic trees
import dendropy
import torch

def sparse2pairs(sparsemat, matrows = None):
    '''
    This functino takes a sparse matrix and returns a list of pairs of the non zero entries
    args:
        sparsemat: a sparse matrix
        matrows: a list of the matrix rows to keep
    Returns:    
        a list of pairs of the non zero entries
    '''
    if matrows :
        sparsemat = sparsemat[matrows,:]
        sparsemat = sparsemat[:,matrows]
    sparsemat = scipy.sparse.find(sparsemat)
    return np.vstack([sparsemat[0],sparsemat[1]])


class PhyloTreeEnvironment:
    def __init__(self , newick_tree):
        self.state = self.reset()
        self.action_dim = 4  # Number of nodes + 1 for the "end" action
        self.newick_tree = newick_tree
        # Define the initial graph state
        self.tree = dendropy.Tree.get(data=newick_tree, schema="newick")
        #label the nodes with a matrix row index
        for i, node in enumerate(self.tree.preorder_node_iter()):
            node.matrow = i
        N = len(tree.nodes())
        #mimic the fitch algo
        #propagate up and down in separate graphs
        index_up = np.vstack([ [n.matrow, c.matrow ] for n in tree.nodes() for c in n.child_nodes()])
        index_down = np.vstack([ [c.matrow, n.matrow ] for n in tree.nodes() for c in n.child_nodes()])
        connectmat_up = scipy.sparse.lil_matrix(( N ,  N ) )
        connectmat_down = scipy.sparse.lil_matrix(( N ,  N ) )
        connectmat_up[index_up[:,0],index_up[:,1]] = 1 
        connectmat_down[index_down[:,0],index_down[:,1]] = 1 
        diag = [[n,n] for n in range(N)]
        connectmat_diag=scipy.sparse.lil_matrix(( N ,  N ) )
        connectmat_diag[diag,diag] = 1 
        #create a pytorch geometric graph
        self.connectmat_down = torch.tensor( sparse2pairs(connectmat_down) , dtype=torch.long) 
        self.connectmat_up = torch.tensor( sparse2pairs(connectmat_up) , dtype=torch.long)
        self.connectmat_diag = torch.tensor( sparse2pairs(connectmat_diag) , dtype=torch.long)

    def tree2graph(self):
        index_up = np.vstack([ [n.matrow, c.matrow ] for n in tree.nodes() for c in n.child_nodes()])
        index_down = np.vstack([ [c.matrow, n.matrow ] for n in tree.nodes() for c in n.child_nodes()])
        connectmat_up = scipy.sparse.lil_matrix(( N ,  N ) )
        connectmat_down = scipy.sparse.lil_matrix(( N ,  N ) )
        connectmat_up[index_up[:,0],index_up[:,1]] = 1 
        connectmat_down[index_down[:,0],index_down[:,1]] = 1 
        diag = [[n,n] for n in range(N)]
        connectmat_diag=scipy.sparse.lil_matrix(( N ,  N ) )
        connectmat_diag[diag,diag] = 1 
        #create a pytorch geometric graph
        self.connectmat_down = torch.tensor( sparse2pairs(connectmat_down) , dtype=torch.long) 
        self.connectmat_up = torch.tensor( sparse2pairs(connectmat_up) , dtype=torch.long)
        self.connectmat_diag = torch.tensor( sparse2pairs(connectmat_diag) , dtype=torch.long)

    def step(self, action):
        #cut and graft a node

        #reinit the graph
        

    def cut_and_grat(self, cutnode , graftnode ):
        #use dendropy to cut and graft a node on a branch

        #cut the node
        parent = cutnode.parent_node
        parent.remove_child(cutnode)
        up_one = parent.parent_node
        #gab the other child of the parent node
        other_child = parent.child_nodes()[0]
        up_one.add_child(other_child)

        #collapse the branch where the node was cut

        #make the dendropy tree from the graph



        #graft the node on the branch leading to the graftnode
        

    def reset(self):
        # Define the initial graph state
        x = torch.tensor([[1], [2], [3]], dtype=torch.float)

    def label_internal_nodes(self , evomodel = None ):
        tree = evomodel(self.tree )



In [2]:
pip install dendropy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.0/459.0 kB 12.8 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [11]:
import dendropy
from dendropy import Tree, Node

def cut_subtree(tree, node_label):
    """
    Cut a subtree from a given tree at the specified node label.
    
    Args:
        tree (Tree): The tree to cut from.
        node_label (str): The label of the node where the subtree will be cut.
    
    Returns:
        Tree: The subtree that was cut.
        Node: The original node where the subtree was cut.
    """
    node = tree.find_node_with_label(node_label)
    if node is None:
        raise ValueError(f"Node with label {node_label} not found in the tree.")
    
    parent_node = node.parent_node
    if parent_node is None:
        raise ValueError("Cannot cut the root node.")
    
    parent_node.remove_child(node)
    subtree = Tree(seed_node=node)
    
    return subtree, parent_node

def graft_subtree(tree, graft_point_label, subtree):
    """
    Graft a subtree onto a target tree at the specified graft point label while ensuring binary tree constraints.
    
    Args:
        tree (Tree): The tree to graft onto.
        graft_point_label (str): The label of the node where the subtree will be grafted.
        subtree (Tree): The subtree to graft.
    """
    graft_point_node = tree.find_node_with_label(graft_point_label)
    if graft_point_node is None:
        raise ValueError(f"Node with label {graft_point_label} not found in the tree.")
    
    if len(graft_point_node.child_nodes()) < 2:
        graft_point_node.add_child(subtree.seed_node)
    else:
        # Create a new internal node to maintain binary tree structure
        new_internal_node = Node()
        new_internal_node.add_child(subtree.seed_node)
        
        # Move one of the existing children to the new internal node
        existing_child = graft_point_node.child_nodes()[1]
        graft_point_node.remove_child(existing_child)
        new_internal_node.add_child(existing_child)
        
        # Add the new internal node to the graft point node
        graft_point_node.add_child(new_internal_node)

def cut_and_graft(tree, source_node_label, target_node_label):
    """
    Cut a subtree from the tree and graft it onto the same tree while maintaining binary tree structure.
    
    Args:
        tree (Tree): The tree to modify.
        source_node_label (str): The label of the node where the subtree will be cut.
        target_node_label (str): The label of the node where the subtree will be grafted.
    """
    subtree, _ = cut_subtree(tree, source_node_label)
    graft_subtree(tree, target_node_label, subtree)

# Example usage
if __name__ == "__main__":
    tree_newick = "((A,B,(C,D)),(E,(F,G)));"

    tree = Tree.get(data=tree_newick, schema="newick")
    
    
    print("Original Tree:")
    print(tree.as_string(schema="newick"))
    print(tree.as_ascii_plot())
   
    cut_and_graft(tree, "C", "G")
    print("Modified Tree:")
    print(tree.as_string(schema="newick"))


Original Tree:
((A,B,(C,D)),(E,(F,G)));

                         /--------------------------------------------------- A
                         |                                                     
/------------------------+--------------------------------------------------- B
|                        |                                                     
|                        |                         /------------------------- C
+                        \-------------------------+                           
|                                                  \------------------------- D
|                                                                              
|                        /--------------------------------------------------- E
\------------------------+                                                     
                         |                         /------------------------- F
                         \-------------------------+                           

ValueError: Node with label C not found in the tree.

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import deque
import random
from torch_geometric.data import Data

# Define the Graph Neural Network
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# Define the DQN Agent
class DQNAgent:
    def __init__(self, gnn, state_dim, action_dim):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.memory = deque(maxlen=10000)
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        
        self.gnn = gnn
        self.fc = nn.Linear(state_dim, action_dim)
        self.target_gnn = gnn
        self.target_fc = nn.Linear(state_dim, action_dim)
        self.update_target_model()
        
        self.optimizer = optim.Adam(list(self.gnn.parameters()) + list(self.fc.parameters()), lr=self.learning_rate)
        self.criterion = nn.MSELoss()
    
    def update_target_model(self):
        self.target_gnn.load_state_dict(self.gnn.state_dict())
        self.target_fc.load_state_dict(self.fc.state_dict())
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_dim)
        with torch.no_grad():
            state = self.gnn(state)
            q_values = self.fc(state)
        return torch.argmax(q_values).item()
    
    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            with torch.no_grad():
                next_state = self.gnn(next_state)
                target = reward
                if not done:
                    target += self.gamma * torch.max(self.target_fc(next_state)).item()
            state = self.gnn(state)
            target_f = self.fc(state)
            target_f[action] = target
            self.optimizer.zero_grad()
            loss = self.criterion(target_f, self.fc(state))
            loss.backward()
            self.optimizer.step()
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


gnn = GCN(input_dim, hidden_dim, output_dim)
agent = DQNAgent(gnn, output_dim, action_dim)

episodes = 1000
batch_size = 64

for e in range(episodes):
    state = env.reset()
    done = False
    total_reward = 0
    while not done:
        action = agent.act(state)
        next_state, reward, done = env.step(action)
        total_reward += reward
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            agent.update_target_model()
            print(f"Episode {e+1}/{episodes}, Total Reward: {total_reward}, Epsilon: {agent.epsilon:.2}")
        agent.replay(batch_size)
